In [1]:
import soundfile
import pydub
import librosa
import numpy as np
import io
import os
import uuid
from pydub.playback import play

In [2]:
src = 'filter-test-dataset/Backyard-2024-Feb-12-16/0101/050711.wav'
a = soundfile.read(src)
b = librosa.load(src, sr = None)
assert len(a) == 2 and len(a) == len(b)
assert (a[0] == b[0]).all()
assert a[1] == b[1]

In [25]:
def wav_to_mp3_to_wav(wav: np.ndarray, sr: int) -> np.ndarray:
    b = io.BytesIO()
    soundfile.write(b, wav, sr, format = 'WAV')
    b.seek(0)
    seg = pydub.AudioSegment.from_wav(b)
    b = io.BytesIO()
    seg.export(b, format = 'mp3')
    b.seek(0)
    seg = pydub.AudioSegment.from_mp3(b)
    b = io.BytesIO()
    seg.export(b, format = 'wav')
    b.seek(0)
    res = soundfile.read(b)
    res = (librosa.resample(res[0], orig_sr = res[1], target_sr = sr), sr)
    res = (res[0].astype(wav.dtype), res[1])

    assert res[0].dtype == wav.dtype, f'{wav.dtype} -> {res[0].dtype}'
    assert res[0].shape == wav.shape, f'{wav.shape} -> {res[0].shape}'
    assert res[1] == sr, f'{sr} -> {res[1]}'
    return res[0]

def wav_to_mp3_to_wav_fast(wav: np.ndarray, sr: int) -> np.ndarray:
    b = io.BytesIO()
    soundfile.write(b, librosa.resample(wav, orig_sr = sr, target_sr = 44100), 44100, format = 'MP3')
    b.seek(0)
    res = soundfile.read(b)
    res = (librosa.resample(res[0], orig_sr = res[1], target_sr = sr).astype(wav.dtype), sr)
    if res[0].shape == (wav.shape[0] + 1,):
        res = (res[0][:-1], res[1])
    
    assert res[0].dtype == wav.dtype, f'{wav.dtype} -> {res[0].dtype}'
    assert res[0].shape == wav.shape, f'{wav.shape} -> {res[0].shape}'
    assert res[1] == sr, f'{sr} -> {res[1]}'
    return res[0]

x = wav_to_mp3_to_wav(*a)
y = wav_to_mp3_to_wav_fast(*a)
print(a[0])
print(x)
print(y)

[ 0.00158691  0.00640869  0.00585938 ... -0.00079346 -0.00061035
 -0.00048828]
[ 0.00255525  0.00497464  0.00557264 ... -0.00070736 -0.00051302
 -0.00024977]
[ 0.00186842  0.00674783  0.00632671 ... -0.00080003 -0.00069399
 -0.00051579]


In [4]:
play(pydub.AudioSegment(a[0].astype('float32').tobytes(), frame_rate = a[1], sample_width = 4, channels = 1))

ALSA lib pcm_dsnoop.c:641:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave


In [10]:
play(pydub.AudioSegment(x.astype('float32').tobytes(), frame_rate = a[1], sample_width = 4, channels = 1))

ALSA lib pcm_dsnoop.c:641:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave


KeyboardInterrupt: 

In [26]:
play(pydub.AudioSegment(y.astype('float32').tobytes(), frame_rate = a[1], sample_width = 4, channels = 1))

ALSA lib pcm_dsnoop.c:641:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
